In [ ]:
!pip install surprise

     |████████████████████████████████| 11.8 MB 12.6 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1619415 sha256=7b78a25f6fe0a9d32bfd120d3374d964adf5e069b1f5b5b2bc7ba7b6ca0a805f
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

sns.set()
sns.set_style('whitegrid')
sns.set_context('paper', font_scale=1.6, rc={"lines.linewidth": 2})
import matplotlib.pyplot as plt

from surprise import Dataset
from surprise import Reader
from surprise import NormalPredictor
from surprise.model_selection import cross_validate

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#df = pd.read_csv("/content/drive/MyDrive/costumerdata/ratings.csv", dtype='str')
df = pd.read_csv("/content/drive/MyDrive/Analysis of Customer Data/ratings.csv", dtype='str')
df = df.drop(['timestamp'], axis=1)

types_dict = {'userId': int, 'movieId': int, 'rating':float}
for col, col_type in types_dict.items():
    df[col] = df[col].astype(col_type)
    
df

,userId,movieId,rating
0,1,307,3.5
1,1,481,3.5
2,1,1091,1.5
3,1,1257,4.5
4,1,1449,4.5
...,...,...,...
27753439,283228,8542,4.5
27753440,283228,8712,4.5
27753441,283228,34405,4.5
27753442,283228,44761,4.5


In [ ]:
df.dropna(axis='index', inplace=True)
df.head(n=10)

,userId,movieId,rating
0,1,307,3.5
1,1,481,3.5
2,1,1091,1.5
3,1,1257,4.5
4,1,1449,4.5
5,1,1590,2.5
6,1,1591,1.5
7,1,2134,4.5
8,1,2478,4.0
9,1,2840,3.0


*Nuova sezione*

# **Matrix factorization**
The selected method based on matrix factorization is  singular value decomposition (SVD). It's been choosen based on the current resources and dataset. SVDpp which is notoriusly better than SVD has been took into consideration and run but our computational resources weren't enough to run it in a acceptable time and an acceptable amount of data.
The Hyperparameters of SVD are number of epochs, number of factors, learning rate and regularization term. For this assignment we considered changing all of these parameters using GridSearch with 5-fold cross validation. After running we saw that the default current value of 0.005 was the best performing. The hyperparameters optimized that we decided to tune for SVD are: number of epochs and factors and the regularization term:


*   **Epochs**: It is the number of iteration of the SGD (stochastic gradient descent) procedure. We tried values between 10 and 120 the optimal number for our model it's 100, 120 perfomed better but in the order of millesimal.
*   **Factors**: It is used to determine the number of factors to extract/retain during teh singular value decomposition, the default value is 100. We tried values between 50 and 200, the one with the lowest RMSE was 150.
*   **Regularization**: It is used to fit the model appropriately and avoid overfitting. We tried values between 0.5 and 0.02. The best performing regularization term for us was 0.01.

In total with parameters tuning thanks to GridSearch and 5-fold cross validation we achieved a **RMSE** of **0.7955**. Besides comparing it with the Normal predictor we decided to compare it with a baseline SVD with a train-test split of 25% which performed RMSE of 0.8150 and another baseline SVD this time with a 5-fold cross validation which performed a RMSE of 0.8090.



We decided for lack of resources and run the algorithm in a reasonable time to cut the database to the top 100 movies. Attempts with higher numbers (500,1000) when didn't crash didn't give a very high change in performance.

In [ ]:
def select(df, k, col):

    top_values = df[col].value_counts().nlargest(k)
    return df.loc[df[col].isin(top_values.index)]

df = select(df, k=100, col='movieId')

In [ ]:
unique_users = len(df['userId'].unique())
unique_movies = len(df['movieId'].unique())

print('unique users: ' + str(unique_users))
print('unique movies: ' + str(unique_movies))
df

unique users: 236818
unique movies: 100


,userId,movieId,rating
34,3,1221,4.0
40,3,2028,5.0
42,4,1,4.0
46,4,10,4.0
54,4,32,4.5
...,...,...,...
27753419,283228,3578,5.0
27753422,283228,4886,5.0
27753423,283228,4995,4.0
27753429,283228,6377,5.0


In [ ]:
# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(0.5, 5))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)

In [ ]:
#SVD model without cross validation - RMSE .81

from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split


# sample random trainset and testset
# test set is made of 25% of the ratings.
trainset, testset = train_test_split(data, test_size=.25)

# We'll use the famous SVD algorithm.
algo = SVD()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

KeyboardInterrupt: ignored

Exception ignored in: 'zmq.backend.cython.message.Frame.__dealloc__'
Traceback (most recent call last):
  File "zmq/backend/cython/checkrc.pxd", line 13, in zmq.backend.cython.checkrc._check_rc
KeyboardInterrupt


KeyboardInterrupt: ignored

In [ ]:
#svd with cross validation .80

from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate

cross_validate(SVD(),data,measures=['RMSE', 'MAE'],cv=5, verbose=True, n_jobs=-1)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8095  0.8092  0.8096  0.8083  0.8085  0.8090  0.0005  
MAE (testset)     0.6116  0.6120  0.6124  0.6111  0.6103  0.6115  0.0007  
Fit time          454.36  517.72  521.37  457.90  278.97  446.06  88.25   
Test time         15.47   23.85   15.57   15.87   10.32   16.22   4.34    


{'fit_time': (454.36383295059204,
  517.7233657836914,
  521.3688399791718,
  457.89619874954224,
  278.96686363220215),
 'test_mae': array([0.61158103, 0.61200418, 0.61243276, 0.61108795, 0.61032419]),
 'test_rmse': array([0.80950741, 0.80924855, 0.80958064, 0.80830141, 0.80845274]),
 'test_time': (15.470430135726929,
  23.848342657089233,
  15.567516088485718,
  15.874752521514893,
  10.323352813720703)}

In [ ]:
#adding gridsearch with cross-validation
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV

param_grid = {'n_epochs': [100], 'lr_all': [0.005], 'reg_all': [0.2, 0.3], 'n_factors': [100, 150]}

gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=5, n_jobs=-1)

gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

0.8473652819594516
{'n_epochs': 100, 'lr_all': 0.005, 'reg_all': 0.2, 'n_factors': 150}


0.8473652819594516
{'n_epochs': 100, 'lr_all': 0.005, 'reg_all': 0.2, 'n_factors': 150}

In [ ]:
#adding gridsearch with cross-validation
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import RandomizedSearchCV

param_grid = {'n_epochs': [30], 'lr_all': [0.005], 'reg_all': [0.2], 'n_factors': [150], 'biased':[False]}

gs = RandomizedSearchCV(SVD, param_grid, n_iter=1, measures=['rmse', 'mae'], cv=5, n_jobs=-1)

gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

1.231985549913963
{'n_epochs': 30, 'lr_all': 0.005, 'reg_all': 0.2, 'n_factors': 150, 'biased': False}


1.231985549913963
{'n_epochs': 30, 'lr_all': 0.005, 'reg_all': 0.2, 'n_factors': 150, 'biased': False}